<div style="border-bottom:2px solid black; max-width:450px;">
    <h1 >Chessbot with Computer Vision</h1>
</div>

<img src="./images/logo.png" style="width:150px;margin:0 auto;padding:20px 0 20px 0;"/>

<p> En este breve notebook mostraremos el funcionamiento por partes de manera simple e interactiva de nuestro bot de ajedrez basado en visión por computador.<br>Para ilustrar adecuadamente y de forma  amena y comprensible las entrañas del programa dividiremos la explicación en los siguientes apartados:</p>
<ol>
  <li>Captura de pantalla</li>
  <li>Reconocimiento del tablero</li>
  <li>Análisis del mejor movimiento</li>
  <li>Movimiento automático por ordenador</li>
</ol>

<p>Todo el código puede encontrarse en el siguiente enlace <a href="https://github.com/Blackwolf4183/ChessBot_ComputerVision"> en github </a> completo y con su documentación.</p>

<h2 style=""><b>0. Prompt al usuario</b></h2>

<p> Un paso 0, muy trivial pero esencial para nuestro analizador, es preguntar al usuario con que piezas va a jugar la partida (blancas o negras), ya que es imposible determinar dado una configuración aleatoria de tablero sin esta información, que piezas estamos jugando</p>

<h2 style=""><b>1. Captura de pantalla</b></h2>


<p> El primer paso que debemos realizar si queremos que nuestro bot juegue al ajedrez de forma autónoma es alguna forma de darle información primitiva, que en este caso serán capturas de lo que estamos viendo en el ordenador y con ello procesar las imágenes y transformarlas en información útil. <br> Para realizar este paso usaremos la API de windows, la cual podemos acceder a través de la libreria pywin32 y la cual nos permitirá tomar capturas de pantalla, pero también identificar las ventanas activas y maximizarlas o minimizarlas. <br> Dicho esto, el proceso que realizamos en este apartado es el siguiente: </p>

<ol>
  <li>Obtendremos una lista de las ventanas abiertas</li>
    <li>Comprobamos que <a href="www.Chess.com">Chess.com </a> esté abierto en alguna de ellas y no en más de una, puesto que de lo contrario no estaríamos jugando una partida</li>
  <li>Una vez localizada la venta procedemos primero a minimizarla y luego a maximizarla para tenerla en pantalla completa</li>
  <li>Finalmente comienza el bucle en el que se va tomando esporádicamente una captura de lo que hay en pantalla y toca procesarla</li>
</ol>

